# Anomaly Detection in Univariate Time Series with SVR, in the context of WISDom project: Implementation

### Flow Rate Data from a sensor in a Water Sypply System located in Barreiro

# 1. Introduction

## Libraries and Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
# Support vector machine algorithm - supervised learning
from sklearn.preprocessing import StandardScaler
# Standardize features by removing the mean and scaling to unit variance

from sklearn.feature_selection import VarianceThreshold

from sklearn.model_selection import GridSearchCV
# To determinate the best parameters
# Exhaustive search over specified parameter values for an estimator.
# Important members are fit, predict.
# GridSearchCV implements a “fit” and a “score” method. It also implements “score_samples”, “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.
# More in 3.2. Tuning the hyper-parameters of an estimator - https://scikit-learn.org/stable/modules/grid_search.html#grid-search
# And more in 3.1. Cross-validation: evaluating estimator performance - https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
from sklearn.metrics import make_scorer
# Make a scorer (marcador) from a performance metric or loss function
# More in https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html
from sklearn.metrics import mean_squared_error
# Mean squared error function loss
# The mean_squared_error function computes mean square error, a risk metric corresponding to the expected value of the squared (quadratic) error or loss
from math import sqrt
# Inbuilt function that returns the square root of any number
import time
# Module which provides various time-related functions
import seaborn as sns; sns.set()
# For graphics
import math
# Module which provides access to the matematical functions defined by the C standard
from sklearn.metrics import precision_score
# Compute the precision

# 2. Preprocessing

## Load data

In [2]:
# Names of the columns
features = ["date", "time", "flow", "anomaly"]

train = pd.read_csv('C:/Users/Catarina/Documents/WISDom_Internship/WISDom_InternshipCode/WISDom_Internship/Synthetic_Data/RandomForest/Barreiro/train_barreiro_1.csv', sep = ';', names = features)

test = pd.read_csv('C:/Users/Catarina/Documents/WISDom_Internship/WISDom_InternshipCode/WISDom_Internship/Synthetic_Data/RandomForest/Barreiro/test_barreiro_1.csv', sep = ';', names = features)

## 2.1 Feature generation

### Timestamp column

In [3]:
train['timestamp'] = pd.to_datetime(train['time'])

test['timestamp'] = pd.to_datetime(test['time'])

In [4]:
# Train
train['timestamp'] = train['timestamp'].dt.hour * 3600 + \
              train['timestamp'].dt.minute * 60 + \
              train['timestamp'].dt.second
# Test
test['timestamp'] = test['timestamp'].dt.hour * 3600 + \
              test['timestamp'].dt.minute * 60 + \
              test['timestamp'].dt.second

### Weekday column

In [5]:
# Train
train['weekday'] = pd.to_datetime(train['date'],dayfirst=True)
train['weekday'] = train['weekday'].dt.dayofweek
# Test
test['weekday'] = pd.to_datetime(test['date'],dayfirst=True)
test['weekday'] = test['weekday'].dt.dayofweek

### Hour column

In [6]:
# Train
train['hour'] = pd.to_datetime(train['time'],dayfirst=True)
train['hour'] = train['hour'].dt.hour
# Test
test['hour'] = pd.to_datetime(test['time'],dayfirst=True)
test['hour'] = test['hour'].dt.hour

### Datetime column

In [7]:
train['datetime'] = train['date'] + ' ' + train['time']
train['datetime'] = pd.to_datetime(train['datetime'], dayfirst=True).dt.strftime('%Y-%m-%d %H:%M:%S').astype(str)
# Immutable ndarray-like of datetime64 data
# Represented internally as int64, and which can be boxed to Timestamp objects that are subclasses of datetime and carry metadata.
train['datetime'] = pd.DatetimeIndex(data=train['datetime'], dtype='datetime64[ns]', name='datetime', freq=None)

test['datetime'] = test['date'] + ' ' + test['time']
test['datetime'] = pd.to_datetime(test['datetime'], dayfirst=True).dt.strftime('%Y-%m-%d %H:%M:%S').astype(str)
# Immutable ndarray-like of datetime64 data
# Represented internally as int64, and which can be boxed to Timestamp objects that are subclasses of datetime and carry metadata.
test['datetime'] = pd.DatetimeIndex(data=test['datetime'], dtype='datetime64[ns]', name='datetime', freq=None)

In [8]:
# .info() function is used to get a concise summary of the DataFrame
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       480 non-null    object        
 1   time       480 non-null    object        
 2   flow       480 non-null    float64       
 3   anomaly    480 non-null    int64         
 4   timestamp  480 non-null    int64         
 5   weekday    480 non-null    int64         
 6   hour       480 non-null    int64         
 7   datetime   480 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 30.1+ KB


### Final datasets

In [9]:
train

,date,time,flow,anomaly,timestamp,weekday,hour,datetime
0,03/05/2018,00:07:30,13.026578,0,450,3,0,2018-05-03 00:07:30
1,03/05/2018,00:22:30,10.898906,0,1350,3,0,2018-05-03 00:22:30
2,03/05/2018,00:37:30,11.048772,0,2250,3,0,2018-05-03 00:37:30
3,03/05/2018,00:52:30,10.641706,0,3150,3,0,2018-05-03 00:52:30
4,03/05/2018,01:07:30,10.452578,0,4050,3,1,2018-05-03 01:07:30
...,...,...,...,...,...,...,...,...
475,09/05/2018,22:52:30,22.357783,0,82350,2,22,2018-05-09 22:52:30
476,09/05/2018,23:07:30,19.707922,0,83250,2,23,2018-05-09 23:07:30
477,09/05/2018,23:22:30,19.546844,0,84150,2,23,2018-05-09 23:22:30
478,09/05/2018,23:37:30,21.068406,0,85050,2,23,2018-05-09 23:37:30


In [10]:
test

,date,time,flow,anomaly,timestamp,weekday,hour,datetime
0,10/05/2018,00:07:30,15.713694,0,450,3,0,2018-05-10 00:07:30
1,10/05/2018,00:22:30,14.460300,0,1350,3,0,2018-05-10 00:22:30
2,10/05/2018,00:37:30,14.975789,0,2250,3,0,2018-05-10 00:37:30
3,10/05/2018,00:52:30,14.658967,0,3150,3,0,2018-05-10 00:52:30
4,10/05/2018,01:07:30,11.786478,0,4050,3,1,2018-05-10 01:07:30
...,...,...,...,...,...,...,...,...
91,10/05/2018,22:52:30,23.881589,0,82350,3,22,2018-05-10 22:52:30
92,10/05/2018,23:07:30,21.902367,0,83250,3,23,2018-05-10 23:07:30
93,10/05/2018,23:22:30,23.582700,0,84150,3,23,2018-05-10 23:22:30
94,10/05/2018,23:37:30,23.657244,0,85050,3,23,2018-05-10 23:37:30


## Time series datasets

In [11]:
# Group DataFrame using a mapper or by a Series of columns
df = train.groupby('datetime')['flow'].sum().reset_index()

dfT = test.groupby('datetime')['flow'].sum().reset_index()

In [12]:
df

,datetime,flow
0,2018-05-03 00:07:30,13.026578
1,2018-05-03 00:22:30,10.898906
2,2018-05-03 00:37:30,11.048772
3,2018-05-03 00:52:30,10.641706
4,2018-05-03 01:07:30,10.452578
...,...,...
475,2018-05-09 22:52:30,22.357783
476,2018-05-09 23:07:30,19.707922
477,2018-05-09 23:22:30,19.546844
478,2018-05-09 23:37:30,21.068406


## 2.2 Feature selection

### Manually

In [13]:
train = pd.concat([train['weekday'],train['hour'],train['flow'],train['anomaly']], axis=1, join='inner')
test = pd.concat([train['weekday'],train['hour'],train['flow'],train['anomaly']], axis=1, join='inner')

In [14]:
train

,weekday,hour,flow,anomaly
0,3,0,13.026578,0
1,3,0,10.898906,0
2,3,0,11.048772,0
3,3,0,10.641706,0
4,3,1,10.452578,0
...,...,...,...,...
475,2,22,22.357783,0
476,2,23,19.707922,0
477,2,23,19.546844,0
478,2,23,21.068406,0


### Based on VarianceThreshold

In [22]:
sel = VarianceThreshold()
df_train = sel.fit_transform(train)
df_train

array([[ 3.        ,  0.        , 13.02657778],
       [ 3.        ,  0.        , 10.89890556],
       [ 3.        ,  0.        , 11.04877222],
       ...,
       [ 2.        , 23.        , 19.54684444],
       [ 2.        , 23.        , 21.06840556],
       [ 2.        , 23.        , 17.95307222]])

In [23]:
df_test = sel.fit_transform(test)
df_test

array([[ 3.        ,  0.        , 13.02657778],
       [ 3.        ,  0.        , 10.89890556],
       [ 3.        ,  0.        , 11.04877222],
       ...,
       [ 2.        , 23.        , 19.54684444],
       [ 2.        , 23.        , 21.06840556],
       [ 2.        , 23.        , 17.95307222]])

# 3. Data Modelling

## Lags of time series

Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

The simplest approach is to predict the value at the next time (t+1) given the value at the previous time (t-1). The supervised learning problem with shifted values

### 10 previous readings

In [24]:
# Adding the lag of the target variable from 10 steps back up to 24
for i in range(0, 11):
    df["lag_{}".format(i)] = df.flow.shift(i)

In [25]:
df.head(50)

,datetime,flow,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,...,lag_0,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9
0,2018-05-03 00:07:30,13.026578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.026578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-03 00:22:30,10.898906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.898906,13.026578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-03 00:37:30,11.048772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.048772,10.898906,13.026578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-03 00:52:30,10.641706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.641706,11.048772,10.898906,13.026578,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-03 01:07:30,10.452578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.452578,10.641706,11.048772,10.898906,13.026578,NaN,NaN,NaN,NaN,NaN
5,2018-05-03 01:22:30,8.923089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.923089,10.452578,10.641706,11.048772,10.898906,13.026578,NaN,NaN,NaN,NaN
6,2018-05-03 01:37:30,8.638289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.638289,8.923089,10.452578,10.641706,11.048772,10.898906,13.026578,NaN,NaN,NaN
7,2018-05-03 01:52:30,8.334467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.334467,8.638289,8.923089,10.452578,10.641706,11.048772,10.898906,13.026578,NaN,NaN
8,2018-05-03 02:07:30,8.100783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.100783,8.334467,8.638289,8.923089,10.452578,10.641706,11.048772,10.898906,13.026578,NaN
9,2018-05-03 02:22:30,8.706250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.706250,8.100783,8.334467,8.638289,8.923089,10.452578,10.641706,11.048772,10.898906,13.026578


In [17]:
ta = df.flow.shift(periods=10)

In [18]:
ta.head(30)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10    13.026578
11    10.898906
12    11.048772
13    10.641706
14    10.452578
15     8.923089
16     8.638289
17     8.334467
18     8.100783
19     8.706250
20    12.435539
21     9.029189
22     7.252783
23     9.600289
24     7.778017
25     6.946067
26     7.736522
27     7.166511
28     7.321006
29     7.279333
Name: flow, dtype: float64

## Rolling window

In [ ]:
n = 8

# EXEMPLO

Exemplo: TREINO
train = 9h, day 09/05
dados = D=10 antes das 9h dos dias: 09/05 (10 valores), 08/05 (10 valores), 07/05 (10 valores), ...

In [ ]:
X_train = # D=10 antes das 9h de 5 dias

In [ ]:
y_train = # 9h 

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=1, test_size=2)
tscv

In [ ]:
for train, test in tscv.split(X):
    print("TRAIN:", train, "TEST:", test)
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

In [ ]:
svr_rbf = svm.SVR(kernel='rbf', C=1e2, gamma='scale')

In [ ]:
svr_rbf.fit(X_train,y_train)

Implementing SVR

In [ ]:
def predict_ts(X_train, y_train, X_test, y_test):

    svr_rbf = svm.SVR(kernel='rbf', C=1e2, gamma=0.1)
    svr_rbf.fit(X_train,y_train)
    p1 = svr_rbf.predict(X_test)
    
    error = mean_squared_error(y_test, p1)
    print('RBF model Test MSE: %.3f' % error)
    
    y_test = y_test * 20450.83322 + 4975.270898
    p1 = p1 * 20450.83322 + 4975.270898
    
    plt.plot(y_test, label='True Data')
    plt.plot(p1, label='RBF model Predict')
    plt.title('Support Vector Regression(RBF)')
    plt.legend()
    plt.show()

Diogo

In [ ]:
# Gamma is the kernel coefficient for 'rbf', 'poly' or 'sigmoid'
# gamma = scale (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma
gamma = 'scale'
# Parameter ε controls the width of the ε-insensitive zone, used to fit the training data.
# The value of ε can affect the number of support vectors used to construct the regression function.
# The bigger ε, the fewer support vectors are selected. On the other hand, bigger ε-values results in more flat estimates
epsilon_val = 10 # margem de erro do algoritmo
epsilon = [] # guardar todos os valores da margem que damos ao regressor

for i in range(len(X)):
    # std compute the standard deviation along the specified axis
    epsilon.append(epsilon_val * np.std(y[i]))

In [ ]:
svr_rbf = []
for i in range(len(X)):
    svr_rbf.append(svm.SVR(kernel='rbf', epsilon=0.5, gamma='scale'))
    #svr_rbf.append(svm.SVR(kernel='rbf', epsilon=epsilon[i], gamma='scale',C=C[i]))

In [ ]:
def fit_model(X,y,svr):
    regressor = svr.fit(X,y)
    y_pred = regressor.predict(X)
    return regressor, y_pred

In [ ]:
regressor = []
    
for i in range(len(X)):
    # Reshape gives a new shape to an array without changing its data
    # (-1,1) its the format of the matrix of array
    regressor.append(fit_model(np.array(X[i]).reshape(-1,1),np.array(y[i]),svr_rbf[i]))

In [ ]:
y_pred = []
    
for i in range(len(X)):
    y_pred.append((regressor[i][1]))

In [ ]:
plus_epsilon = []
minus_epsilon = []

for i in range(len(X)):
    plus_epsilon.append(epsilon_val + y_pred[i])
    minus_epsilon.append(y_pred[i] - epsilon_val)